In [54]:
import numpy as np
import matplotlib.pyplot as plt
import numpy_financial as npf
import pandas as pd
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import math
from dateutil.relativedelta import relativedelta
import os
from scipy.stats import norm

In [ ]:
data_folder_name = 'Data'
instruments_folder_name = "Instruments"

In [153]:
def return_conversion (r,n,this_period = 1):
    return (1+r)**(n/this_period) - 1

In [2]:
def to_currency (value, multiplier=100, add_currency_symbol=True, currency_symbol = '₹',rounding=0):
    negative_sign = ""
    if value < 0:
        negative_sign = "-"
        value = abs(value)
    value = round(value,rounding)
    decimal_value = round(value%1,rounding)
    value = int(round((value-decimal_value),0))
    if decimal_value == 0:
        currency_string = "" 
    else:
        currency_string = "." + str(decimal_value)[2:]
    base = 1_000
    set = value%base
    currency_string = str(int(round(set,0))) + currency_string
    if (len(str(set)) < len(str(value))) & (len(str(set)) < len(str(base))-1):
        zeros = "".join(['0' for _ in range(len(str(base))-1-len(str(set)))])
        currency_string = zeros + currency_string
        
    converted_value = set
    if set == value:
        if add_currency_symbol:
            return currency_symbol + " " + negative_sign + currency_string
        else: 
            return negative_sign + currency_string
    else:
        while converted_value != value:
            base = base * multiplier
            set = int(round((value%base - converted_value)/base * multiplier,0))
            currency_string = str(int(set)) + "," + currency_string
            if (len(str(int(round(set*base/multiplier,0)))) < len(str(value))) & (len(str(set)) < len(str(multiplier))-1):
                zeros = "".join(['0' for _ in range(len(str(multiplier))-1-len(str(set)))])
                currency_string = zeros + currency_string
            converted_value = int(round(converted_value + (set*base/multiplier),0))

    if add_currency_symbol:
        return currency_symbol + " " + negative_sign + currency_string
    else: 
        return negative_sign + currency_string

to_currency(-78361.78264643789, multiplier=100,rounding=0, add_currency_symbol=True)

'₹ -78,362'

In [3]:
class Position:
    def __init__(self, instrument, date):
        self.instrument = instrument
        self.entry_date = date
        self.pnl = 0
        self.current_value = self.instrument.get_value(date)
        self.entry_value = self.current_value
        self.exit_value = self.current_value
        self.tax_liability = 0
    
    def __repr__ (self):
        return f'''{self.instrument} @ {self.current_value}, exit @ {self.exit_value}, bought @ {self.entry_value}'''
    def __str__ (self):
        return f'''{self.instrument} @ {self.current_value}, exit @ {self.exit_value}, bought @ {self.entry_value}'''
    
    
    def update_params(self,date):
        self.current_value = self.instrument.get_value(date)
        self.pnl = self.current_value - self.entry_value
        
        #Computing tax Component
        holding_period_years = relativedelta(date,self.entry_date).years
        if instrument.type == 'equity':
            if holding_period_years >=1: #Long Term
                self.tax_liability = self.pnl * .1
            else: #Short Term
                self.tax_liability = self.pnl * .15
        elif instrument.type == 'debt':
            if holding_period_years >=3: #long Term
                self.tax_liability = self.pnl * .2
            else: #short term
                self.tax_liability = self.pnl * .3
        self.exit_value = self.current_value - self.tax_liability


            
        




In [96]:
class Instrument:
    def __init__(self,name,ticker,type):
        self.name = name
        self.ticker = ticker
        self.type = type
        self.historical = pd.DataFrame()
    
    def __repr__ (self):
        return f'''{self.name}'''
    def __str__ (self):
        return f'''{self.name}'''
    
    def fetch_historical(self,folder_path,file_name):
        self.historical = pd.read_csv(os.path.join(folder_path, file_name))
        self.historical['Date'] = pd.to_datetime(self.historical['Date'])
        self.historical.sort_values(by='Date', inplace=True)
        self.historical['change'] = (self.historical['Close'] \
            - self.historical['Close'].shift(periods=1)) \
            / self.historical['Close'].shift(periods=1)
        # self.historical['std'] = self.historical['change'].rolling(250).std()
    
    def get_value(self, date):
        return self.historical[self.historical['Date']<=date].iloc[-1]['Close']
    
    def get_fall_prob (self, fall,date, lookback_years = None):
        if not lookback_years:
            lookback_years = 10
        dist_df = self.historical[self.historical['Date']<=date]\
            .iloc[-(lookback_years*250):]
        dist_prob = dist_df[dist_df['change']<=fall].shape[0] / dist_df.shape[0]
        norm_prob = norm(dist_df['change'].mean(), dist_df['change'].std()).cdf(fall)
        return dist_prob if dist_prob > norm_prob else norm_prob
    
    def get_mean_return (self,date, lookback_years = None):
        if not lookback_years:
            lookback_years = 10
        return self.historical[self.historical['Date']<=date]\
            .iloc[-(lookback_years*250):]['change'].mean()

i = Instrument('N','N5','equity')
i.fetch_historical(os.path.join(data_folder_name,instruments_folder_name),'NIFTY50_data.csv')

In [165]:
class Goal:
    def __init__ (self,name, maturity_date,inception_date,
        confidence,maturity_value,min_maturity_value,pmt=0):
        self.name = name
        self.maturity_date = maturity_date
        self.inception_date = inception_date
        self.pmt = pmt
        self.confidence = confidence
        self.maturity_value = maturity_value
        self.min_maturity_value = min_maturity_value
        self.cashflows = pd.DataFrame(index=pd.date_range(start=inception_date, end=maturity_date))\
            .reset_index().rename(columns={'index':'date'})
        self.cashflows['downpayment'] = 0
        self.cashflows['spend'] = 0
        self.cashflows['emi'] = 0
        self.cashflows.loc[self.cashflows['date']==inception_date,'downpayment'] = pmt
        self.cashflows.loc[self.cashflows['date']==maturity_date,'spend'] = maturity_value
        self.emi = self.calculate_emi(rf=return_conversion(.06,1,250))

    def __repr__ (self):
        return f'''{self.name} | {to_currency(self.maturity_value)} on {self.maturity_date.strftime('%d %b %Y')} | EMI: {to_currency(self.emi)}, Downpayment: {to_currency(self.pmt)}'''
    def __str__ (self):
        return self.__repr__()
        
    def calculate_emi(self,rf,payment_day=1):
        tolerance = 1
        emi_guess = npf.pmt(rate = rf, nper=(self.maturity_date-self.inception_date).days/30, 
                        fv=-1*self.maturity_value, pv=self.pmt, when='begin')
        surplus = 0

        while not((surplus>0)&(surplus<tolerance)):
            emi_guess = emi_guess - surplus/((self.maturity_date-self.inception_date).days/30)
            self.cashflows.loc[self.cashflows['date'].dt.day==payment_day,'emi'] = emi_guess
            self.cashflows['cashflow'] = self.cashflows['downpayment'] + self.cashflows['emi'] - self.cashflows['spend']
            surplus = npf.npv(rf,self.cashflows['cashflow'])
            
        self.cashflows.loc[self.cashflows['date'].dt.day==payment_day,'emi'] = emi_guess
        self.cashflows['cashflow'] = self.cashflows['downpayment'] + self.cashflows['emi'] - self.cashflows['spend']
        return emi_guess


In [167]:
e = Goal(name='Eu',maturity_date=dt(2023,8,1),inception_date=dt(2022,2,1),pmt=1_000,confidence=0,maturity_value=2_00_000, min_maturity_value=1_60_000)
print(e)

Eu | ₹ 2,00,000 on 01 Aug 2023 | EMI: ₹ 9,812, Downpayment: ₹ 1,000


In [ ]:


instruments = [{'name':'NIFTY 50','filename':'NIFTY50_Data.csv'},
                {'name':'NIFTY High Beta','filename':'NIFTY HIGH BETA 50_Data.csv'},
                {'name':'NIFTY 500','filename':'NIFTY 500_Data.csv'},
                {'name':'NIFTY BANK','filename':'NIFTY BANK_Data.csv'}]

for instrument in instruments:
    instrument['df'] = pd.read_csv(os.path.join(data_folder_name, instruments_folder_name, instrument['filename']))
    instrument['df']['Date'] = pd.to_datetime(instrument['df']['Date'])
    instrument['df'].sort_values(by='Date', inplace=True)
    instrument['df']['change'] = (instrument['df']['Close'] - instrument['df']['Close'].shift(periods=1)) / instrument['df']['Close'].shift(periods=1)
    print(f'''{instrument['name']}\t mu = {round(instrument['df']['change'].mean()*100,2)}%\t sigma = {round(instrument['df']['change'].std()*100,2)}%''')

np.random.seed(43)